# CARAI (Automated Learning and Inference for Causal Effects)

- Setup;
- Import tables;
- Transform;
- Export `.csv`.

# 0. Setup

In [38]:
#standard packages
import pandas as pd
import numpy as np
import os
#import warnings
import warnings
warnings.filterwarnings('ignore')

#machine-learning packages
#from sklearn.preprocessing import LabelEncoder,label_binarize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, auc, accuracy_score
#from imblearn.over_sampling import SMOTE
#from imblearn.pipeline import Pipeline

#image, visualization
import matplotlib.pyplot as plt
import seaborn as sns

***

# 1. Import transformed table & pre-(pre-process)

In [39]:
sm_df = pd.read_csv('/Users/luiz.superti/Documents/GitHub/alice-coding/ML-models/cousin-itt/csv/alice_sm_train_df.csv')
sm_df.drop(columns = 'member_internal_code', inplace = True)

sm_df['daly_t0'].fillna(0, inplace = True)
sm_df['daly_t0'] = sm_df['daly_t0']
sm_df['daly_t1'].fillna(0, inplace = True)
sm_df['daly_t1'] = sm_df['daly_t1']

sm_df['delta_daly'] = sm_df['daly_t1'] - sm_df['daly_t0']
sm_df.loc[sm_df['treatment_group']==0, 'discharged'] = 1 - sm_df['discharged']
sm_df.head(5)

,program_name,program_severity,is_male,is_trans,member_age,total_mths,psico_qt_consultas,psiqui_qt_consultas,discharged,psico_custos_tot,psiqui_custos_tot,treatment_group,daly_t0,daly_t1,delta_daly
0,Ansiedade LEVE,leve,1,0,35.0,14,9.0,0.0,0,0.0,0.0,1,0.030,0.030,0.000
1,Ansiedade MODERADA,moderada,1,0,36.0,3,1.0,0.0,0,150.0,0.0,1,0.030,0.030,0.000
2,Ansiedade LEVE,leve,0,0,32.0,8,7.0,0.0,0,560.0,0.0,1,0.030,0.523,0.493
3,Transtorno Tipo 1,tipo 1,0,0,33.0,4,5.0,0.0,1,630.0,0.0,1,0.523,0.000,-0.523
4,Transtorno Tipo 1,tipo 1,0,0,26.0,6,6.0,3.0,1,750.0,1050.0,1,0.000,0.000,0.000


In [40]:
sm_df['psico_qt_consultas'].mean()

2.8719886632026452

In [41]:
sm_df['psico_custos_tot'].sum()

1166133.7684210525

In [42]:
np.percentile(sm_df[sm_df['treatment_group']==1]['psico_qt_consultas'], 50)

4.0

In [43]:
sm_df[sm_df['treatment_group']==1]['daly_t0'].describe()

count    1213.000000
mean        0.142448
std         0.211542
min         0.000000
25%         0.030000
50%         0.030000
75%         0.145000
max         1.381000
Name: daly_t0, dtype: float64

In [44]:
### Treatment
sm_df['base_control'] = np.where(sm_df['treatment_group']==0,1,0)
sm_df['treatment_0_4'] = np.where((sm_df['treatment_group']==1) & (sm_df['psico_qt_consultas']>=4) & (sm_df['psico_qt_consultas']<=7),1,0)
sm_df['treatment_5_10'] = np.where((sm_df['treatment_group']==1) & (sm_df['psico_qt_consultas']>=8) & (sm_df['psico_qt_consultas']<=11),1,0)
sm_df['treatment_11_'] = np.where((sm_df['treatment_group']==1) & (sm_df['psico_qt_consultas']>11),1,0)
sm_df['treatment_0_10'] = np.where((sm_df['treatment_group']==1) & (sm_df['psico_qt_consultas']>=0) & (sm_df['psico_qt_consultas']<=10),1,0)

In [45]:
ml_df = sm_df[((sm_df['program_severity'].isna()==True)|(sm_df['program_severity']=='tipo 1')|(sm_df['program_severity']=='tipo 2'))][['member_age','is_male','daly_t0', 'delta_daly','base_control','treatment_0_4','psico_custos_tot','treatment_0_10', 'treatment_5_10', 'treatment_11_', 'discharged', 'total_mths']]

In [46]:
ml_df = ml_df[ml_df['is_male']==1]

In [47]:
ml_df.shape

(588, 12)

In [48]:
#probability weights for ATE and cATE
P_D_ = 1-np.mean(ml_df.base_control)
p_treat_0_4 = np.mean(ml_df.treatment_0_4)/(1-np.mean(ml_df.base_control))
p_treat_5_10 = np.mean(ml_df.treatment_5_10)/(1-np.mean(ml_df.base_control))
p_treat_11_ = np.mean(ml_df.treatment_11_)/(1-np.mean(ml_df.base_control))

In [49]:
p_treat_0_10 = np.mean(ml_df.treatment_0_10)/(1-np.mean(ml_df.base_control))

In [50]:
print(p_treat_0_4,p_treat_5_10, p_treat_11_)

0.4517766497461929 0.18274111675126903 0.04568527918781726


In [51]:
p_treat_0_10

0.9238578680203047

In [52]:
from setup import main as setup
setup(env='vscode', project='cousin-itt')
from src.model import DrDidEstimator
dr_did = DrDidEstimator()

Project directory: /Users/luiz.superti/Documents/GitHub/alice-coding/ML-models/cousin-itt


FULL ATE

In [53]:
#cATT(d|d), for d = treat_0_10
#We have to restrict base control and treatment_0_10
catt_dd_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_0_10']==1)]
catt_dd_df['treat'] = np.where(catt_dd_df.treatment_0_10==1,1,0)

care_cost_dd = np.sum(-catt_dd_df[catt_dd_df['discharged']==1]['delta_daly'])/np.sum(catt_dd_df[catt_dd_df['treat']==1]['psico_custos_tot'])

#catt_da_df
#We have to restrict the first alternative treatment a, and treatment_5_10, but ask for the base_control
dr_did.fit(catt_dd_df[['member_age', 'is_male']].to_numpy(),catt_dd_df.treat, catt_dd_df.discharged) 
catt_dd = dr_did.att_mean
catt_dd_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_dd_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)


IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True


In [54]:
print(catt_dd_lb, catt_dd, catt_dd_ub)

0.7619847716157708 0.8164883602387838 0.8709919488617968


#### ATE for d = 5 to 8

In [55]:
#cATT(d|d), for d = treat_5_10
#We have to restrict base control and treatment_5_10
catt_dd_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_5_10']==1)]
catt_dd_df['treat'] = np.where(catt_dd_df.treatment_5_10==1,1,0)

care_cost_dd = np.sum(-catt_dd_df[catt_dd_df['discharged']==1]['delta_daly'])/np.sum(catt_dd_df[catt_dd_df['treat']==1]['psico_custos_tot'])

#catt_da_df
#We have to restrict the first alternative treatment a, and treatment_5_10, but ask for the base_control
catt_aa_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_0_4']==1)]
catt_aa_df['treat']  = np.where(catt_aa_df.treatment_0_4==1,1,0)
catt_ad_df = ml_df[(ml_df['treatment_0_4']==1)|(ml_df['treatment_5_10']==1)]
catt_ad_df['treat']  = np.where(catt_ad_df.treatment_5_10==1,1,0)

#care-costs a/d
nd = len(catt_ad_df[catt_ad_df['treat']==1]['treat'])
cc_d = np.mean(catt_ad_df[catt_ad_df['treat']==1]['psico_custos_tot'])
cc_a = np.mean(catt_ad_df[catt_ad_df['treat']==0]['psico_custos_tot'])

care_cost_da = np.sum(-catt_ad_df[catt_ad_df['treat']==1]['delta_daly'])/(nd*cc_d) - np.sum(catt_ad_df[catt_ad_df['treat']==1]['delta_daly'])/(nd*cc_a)

#We have to restrict the first alternative treatment b, and treatment_5_10, but ask for the base_control
catt_bb_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_11_']==1)]
catt_bb_df['treat'] = np.where(catt_bb_df.treatment_11_==1,1,0)
catt_bd_df = ml_df[(ml_df['treatment_11_']==1)|(ml_df['treatment_5_10']==1)]
catt_bd_df['treat']  = np.where(catt_bd_df.treatment_5_10==1,1,0)

#care-costs b/d
nd = len(catt_bd_df[catt_bd_df['treat']==1]['treat'])
cc_d = np.mean(catt_bd_df[catt_bd_df['treat']==1]['psico_custos_tot'])
cc_b = np.mean(catt_bd_df[catt_bd_df['treat']==0]['psico_custos_tot'])

care_cost_db = np.sum(-catt_bd_df[catt_bd_df['treat']==1]['delta_daly'])/(nd*cc_d) - np.sum(catt_bd_df[catt_bd_df['treat']==1]['delta_daly'])/(nd*cc_b)

#catt_dd
dr_did.fit(catt_dd_df[['member_age', 'is_male']].to_numpy(),catt_dd_df.treat, catt_dd_df.discharged) 
catt_dd = p_treat_5_10*dr_did.att_mean
catt_dd_lb = p_treat_5_10*(dr_did.att_mean - 1.96*dr_did.att_sd)
catt_dd_ub = p_treat_5_10*(dr_did.att_mean + 1.96*dr_did.att_sd)

#catt_da
dr_did.fit(catt_aa_df[['member_age', 'is_male']].to_numpy(),catt_aa_df.treat, catt_aa_df.discharged) 
catt_aa = dr_did.att_mean
catt_aa_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_aa_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)
dr_did.fit(catt_ad_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_ad_df.treat, catt_ad_df.discharged) 
catt_ad = dr_did.att_mean
catt_ad_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_ad_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)
catt_da = p_treat_0_4*(catt_aa-catt_ad)
catt_da_lb = p_treat_0_4*np.fmin((catt_aa_lb-catt_ad_ub),(catt_aa_lb - catt_ad_lb))
catt_da_ub = p_treat_0_4*np.fmax((catt_aa_ub-catt_ad_ub),(catt_aa_ub - catt_ad_lb)) 

#catt_db
dr_did.fit(catt_bb_df[['member_age', 'is_male']].to_numpy(),catt_bb_df.treat, catt_bb_df.discharged) 
catt_bb = dr_did.att_mean
catt_bb_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_bb_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)
dr_did.fit(catt_bd_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_bd_df.treat, catt_bd_df.discharged) 
catt_bd = dr_did.att_mean
catt_bd_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_bd_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)
catt_db = p_treat_11_*(catt_bb-catt_bd)
catt_db_lb = p_treat_11_*np.fmin((catt_bb_lb-catt_bd_ub),(catt_bb_lb - catt_bd_lb))
catt_db_ub = p_treat_11_*np.fmax((catt_bb_ub-catt_bd_ub),(catt_bb_ub - catt_bd_lb))

##############
ate_5_10 = catt_dd + catt_da + catt_db
ate_5_10_lb = catt_dd_lb + catt_da_lb + catt_db_lb
ate_5_10_ub = catt_dd_ub + catt_da_ub + catt_db_ub

cost_eff_5_10 = 10000*(care_cost_dd*catt_dd + care_cost_da*catt_da + care_cost_db*catt_db)
cost_eff_5_10_lb = 10000*(care_cost_dd*catt_dd_lb + care_cost_da*catt_da_lb + care_cost_db*catt_db_lb)
cost_eff_5_10_ub = 10000*(care_cost_dd*catt_dd_ub + care_cost_da*catt_da_ub + care_cost_db*catt_db_ub)

IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True


#### ATE for d = 0 to 4

In [56]:
#cATT(d|d), for d = treat_0_4
#We have to restrict base control and treatment_0_4
catt_dd_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_0_4']==1)]
catt_dd_df['treat'] = np.where(catt_dd_df.treatment_0_4==1,1,0)

care_cost_dd = np.sum(-catt_dd_df[catt_dd_df['discharged']==1]['delta_daly'])/np.sum(catt_dd_df[catt_dd_df['treat']==1]['psico_custos_tot'])

#catt_da_df
#We have to restrict the first alternative treatment a, and d, but ask for the base_control
catt_aa_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_5_10']==1)]
catt_aa_df['treat'] = np.where(catt_aa_df.treatment_5_10==1,1,0)
catt_ad_df = ml_df[(ml_df['treatment_5_10']==1)|(ml_df['treatment_0_4']==1)]
catt_ad_df['treat'] = np.where(catt_ad_df.treatment_0_4==1,1,0)

#care-costs a/d
nd = len(catt_ad_df[catt_ad_df['treat']==1]['treat'])
cc_d = np.mean(catt_ad_df[catt_ad_df['treat']==1]['psico_custos_tot'])
cc_a = np.mean(catt_ad_df[catt_ad_df['treat']==0]['psico_custos_tot'])

care_cost_da = np.sum(-catt_ad_df[catt_ad_df['treat']==1]['delta_daly'])/(nd*cc_d) - np.sum(catt_ad_df[catt_ad_df['treat']==1]['delta_daly'])/(nd*cc_a)

#We have to restrict the first alternative treatment b, and d, but ask for the base_control
catt_bb_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_11_']==1)]
catt_bb_df['treat'] = np.where(catt_bb_df.treatment_11_==1,1,0)
catt_bd_df = ml_df[(ml_df['treatment_11_']==1)|(ml_df['treatment_0_4']==1)]
catt_bd_df['treat'] = np.where(catt_bd_df.treatment_0_4==1,1,0)

#care-costs b/d
nd = len(catt_bd_df[catt_bd_df['treat']==1]['treat'])
cc_d = np.mean(catt_bd_df[catt_bd_df['treat']==1]['psico_custos_tot'])
cc_b = np.mean(catt_bd_df[catt_bd_df['treat']==0]['psico_custos_tot'])

care_cost_db = np.sum(-catt_bd_df[catt_bd_df['treat']==1]['delta_daly'])/(nd*cc_d) - np.sum(catt_bd_df[catt_bd_df['treat']==1]['delta_daly'])/(nd*cc_b)

#catt_dd
dr_did.fit(catt_dd_df[['member_age', 'is_male']].to_numpy(),catt_dd_df.treat, catt_dd_df.discharged) 
catt_dd = p_treat_0_4*dr_did.att_mean
catt_dd_lb = p_treat_0_4*(dr_did.att_mean - 1.96*dr_did.att_sd)
catt_dd_ub = p_treat_0_4*(dr_did.att_mean + 1.96*dr_did.att_sd)

#catt_da
dr_did.fit(catt_aa_df[['member_age', 'is_male']].to_numpy(),catt_aa_df.treat, catt_aa_df.discharged) 
catt_aa = dr_did.att_mean
catt_aa_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_aa_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)
dr_did.fit(catt_ad_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_ad_df.treat, catt_ad_df.discharged) 
catt_ad = dr_did.att_mean
catt_ad_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_ad_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)

catt_da = p_treat_5_10*(catt_aa-catt_ad)
catt_da_lb = p_treat_5_10*np.fmin((catt_aa_lb-catt_ad_ub),(catt_aa_lb - catt_ad_lb))
catt_da_ub = p_treat_5_10*np.fmax((catt_aa_ub-catt_ad_ub),(catt_aa_ub - catt_ad_lb)) 

#catt_db
dr_did.fit(catt_bb_df[['member_age', 'is_male']].to_numpy(),catt_bb_df.treat, catt_bb_df.discharged) 
catt_bb = dr_did.att_mean
catt_bb_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_bb_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)
dr_did.fit(catt_bd_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_bd_df.treat, catt_bd_df.discharged) 
catt_bd = dr_did.att_mean
catt_bd_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_bd_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)

catt_db = p_treat_11_*(catt_bb-catt_bd)
catt_db_lb = p_treat_11_*np.fmin((catt_bb_lb-catt_bd_ub),(catt_bb_lb - catt_bd_lb))
catt_db_ub = p_treat_11_*np.fmax((catt_bb_ub-catt_bd_ub),(catt_bb_ub - catt_bd_lb))


#
ate_0_4 = catt_dd + catt_da + catt_db
cost_eff_0_4 = 10000*(care_cost_dd*catt_dd + care_cost_da*catt_da + care_cost_db*catt_db)  
cost_eff_0_4_lb = 10000*(care_cost_dd*catt_dd_lb + care_cost_da*catt_da_lb + care_cost_db*catt_db_lb)
cost_eff_0_4_ub = 10000*(care_cost_dd*catt_dd_ub + care_cost_da*catt_da_ub + care_cost_db*catt_db_ub)

IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True


#### ATE for d >= 9

In [57]:
#cATT(d|d), for d = treat_11_
#We have to restrict base control and treatment_0_4
catt_dd_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_11_']==1)]
catt_dd_df['treat'] = np.where(catt_dd_df.treatment_11_==1,1,0)

care_cost_dd = np.sum(-catt_dd_df[catt_dd_df['discharged']==1]['delta_daly'])/np.sum(catt_dd_df[catt_dd_df['treat']==1]['psico_custos_tot'])

#catt_da_df
#We have to restrict the first alternative treatment a, and d, but ask for the base_control
catt_aa_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_0_4']==1)]
catt_aa_df['treat'] = np.where(catt_aa_df.treatment_0_4==1,1,0)
catt_ad_df = ml_df[(ml_df['treatment_0_4']==1)|(ml_df['treatment_11_']==1)]
catt_ad_df['treat'] = np.where(catt_ad_df.treatment_11_==1,1,0)


#care-costs a/d
nd = len(catt_ad_df[catt_ad_df['treat']==1]['treat'])
cc_d = np.mean(catt_ad_df[catt_ad_df['treat']==1]['psico_custos_tot'])
cc_a = np.mean(catt_ad_df[catt_ad_df['treat']==0]['psico_custos_tot'])

care_cost_da = np.sum(-catt_ad_df[catt_ad_df['treat']==1]['delta_daly'])/(nd*cc_d) - np.sum(catt_ad_df[catt_ad_df['treat']==1]['delta_daly'])/(nd*cc_a)



#We have to restrict the first alternative treatment b, and d, but ask for the base_control
catt_bb_df = ml_df[(ml_df['base_control']==1)|(ml_df['treatment_5_10']==1)]
catt_bb_df['treat'] = np.where(catt_bb_df.treatment_5_10==1,1,0)
catt_bd_df = ml_df[(ml_df['treatment_5_10']==1)|(ml_df['treatment_11_']==1)]
catt_bd_df['treat'] = np.where(catt_bd_df.treatment_11_==1,1,0)



#care-costs b/d
nd = len(catt_bd_df[catt_bd_df['treat']==1]['treat'])
cc_d = np.mean(catt_bd_df[catt_bd_df['treat']==1]['psico_custos_tot'])
cc_b = np.mean(catt_bd_df[catt_bd_df['treat']==0]['psico_custos_tot'])

care_cost_db = np.sum(-catt_bd_df[catt_bd_df['treat']==1]['delta_daly'])/(nd*cc_d) - np.sum(catt_bd_df[catt_bd_df['treat']==1]['delta_daly'])/(nd*cc_b)

#catt_dd
dr_did.fit(catt_dd_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_dd_df.treat, catt_dd_df.discharged) 
catt_dd = p_treat_11_*dr_did.att_mean
catt_dd_lb = p_treat_11_*(dr_did.att_mean - 1.96*dr_did.att_sd)
catt_dd_ub = p_treat_11_*(dr_did.att_mean + 1.96*dr_did.att_sd)

#catt_da
dr_did.fit(catt_aa_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_aa_df.treat, catt_aa_df.discharged) 
catt_aa = dr_did.att_mean
catt_aa_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_aa_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)

dr_did.fit(catt_ad_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_ad_df.treat, catt_ad_df.discharged) 
catt_ad = dr_did.att_mean
catt_ad_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_ad_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)

catt_da = p_treat_0_4*(catt_aa-catt_ad)
catt_da_lb = p_treat_0_4*np.fmin((catt_aa_lb-catt_ad_ub),(catt_aa_lb - catt_ad_lb))
catt_da_ub = p_treat_0_4*np.fmax((catt_aa_ub-catt_ad_ub),(catt_aa_ub - catt_ad_lb)) 



#catt_db
dr_did.fit(catt_bb_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_bb_df.treat, catt_bb_df.discharged) 
catt_bb = dr_did.att_mean
catt_bd_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_bd_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)

dr_did.fit(catt_bd_df[['member_age', 'is_male', 'daly_t0']].to_numpy(),catt_bd_df.treat, catt_bd_df.discharged) 
catt_bd = dr_did.att_mean
catt_bd_lb = (dr_did.att_mean - 1.96*dr_did.att_sd)
catt_bd_ub = (dr_did.att_mean + 1.96*dr_did.att_sd)

catt_db = p_treat_5_10*(catt_bb-catt_bd)
catt_db_lb = p_treat_5_10*np.fmin((catt_bb_lb-catt_bd_ub),(catt_bb_lb - catt_bd_lb))
catt_db_ub = p_treat_5_10*np.fmax((catt_bb_ub-catt_bd_ub),(catt_bb_ub - catt_bd_lb))
#
ate_11_ = catt_dd + catt_da + catt_db
cost_eff_11_ = 10000*(care_cost_dd*catt_dd + care_cost_da*catt_da + care_cost_db*catt_db)  
cost_eff_11_lb = 10000*(care_cost_dd*catt_dd_lb + care_cost_da*catt_da_lb + care_cost_db*catt_db_lb)
cost_eff_11_ub = 10000*(care_cost_dd*catt_dd_ub + care_cost_da*catt_da_ub + care_cost_db*catt_db_ub)


IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True
IPT estimation started
IPT optimation success: True
WLS estimation started
WLS optimation success: True


#### cost-effectivenss prints

In [58]:
print(cost_eff_0_4_lb,cost_eff_0_4,cost_eff_0_4_ub)

0.9182066984453097 1.068132388198055 1.2180580779508001


In [59]:
print(cost_eff_5_10_lb,cost_eff_5_10,cost_eff_5_10_ub)

0.48069755905022965 0.597325010781185 0.7139524625121404


In [60]:
print(cost_eff_11_lb,cost_eff_11_,cost_eff_11_ub)

nan nan nan


***

# 2. Model & Analysis of cATE
